# OWSLib versus Birdy

This notebook shows a side-by-side comparison of `owslib.wps.WebProcessingService` and `birdy.WPSClient`. 


In [ ]:
from owslib.wps import WebProcessingService

from birdy import WPSClient

url = "https://bovec.dkrz.de/ows/proxy/emu?Service=WPS&Request=GetCapabilities&Version=1.0.0"

wps = WebProcessingService(url)
cli = WPSClient(url=url)

## Displaying available processes

With `owslib`, `wps.processes` is the list of processes offered by the server. 
With `birdy`, the client is like a module with functions. So you just write `cli.` and press `Tab` to display a drop-down menu of processes. 

In [ ]:
wps.processes

## Documentation about a process

With `owslib`, the process title and abstract can be obtained simply by looking at these attributes. 
For the process inputs, we need to iterate on the inputs and access their individual attributes. To facilitate this, `owslib.wps` provides the `printInputOuput` function. 

With `birdy`, just type `help(cli.hello)` and the docstring will show up in your console. With the IPython console or a Jupyter Notebook, `cli.hello?` would do as well. The docstring follows the NumPy convention. 

In [ ]:
from owslib.wps import printInputOutput

p = wps.describeprocess("hello")
print("Title: ", p.title)
print("Abstract: ", p.abstract)

for inpt in p.dataInputs:
    printInputOutput(inpt)

In [ ]:
help(cli.hello)

## Launching a process and retrieving literal outputs

With `owslib`, processes are launched using the `execute` method. Inputs are an an argument to `execute` and defined by a list of key-value tuples. These keys are the input names, and the values are string representations. The `execute` method returns a `WPSExecution` object, which defines a number of methods and attributes, including `isComplete` and `isSucceeded`. The process outputs are stored in the `processOutputs` list, whose content is stored in the `data` attribute. Note that this `data` is a list of strings, so we may have to convert it to a `float` to use it. 

In [ ]:
resp = wps.execute(
    "binaryoperatorfornumbers",
    inputs=[("inputa", "1.0"), ("inputb", "2.0"), ("operator", "add")],
)
if resp.isSucceded:
    (output,) = resp.processOutputs
    print(output.data)

With `birdy`, inputs are just typical keyword arguments, and outputs are already converted into python objects. Since some processes may have multiple outputs, processes always return a `namedtuple`, even in the case where there is only a single output. 

In [ ]:
z = cli.binaryoperatorfornumbers(1, 2, operator="add").get()[0]
z

In [ ]:
out = cli.inout().get()
out.date

## Retrieving outputs by references

For `ComplexData` objects, WPS servers often return a reference to the output (an http link) instead of the actual data. This is useful if that output is to serve as an input to another process, so as to avoid passing back and forth large files for nothing. 

With `owslib`, that means that the `data` attribute of the output is empty, and we instead access the `reference` attribute. The referenced file can be written to the local disk using the `writeToDisk` method. 

With `birdy`, the outputs are by default the references themselves, but it's also possible to download these references in the background and convert them into python objects. To trigger this automatic conversion, set `convert_objects` to `True` when instantating the client `WPSClient(url, convert_objects=True)`. Ini the example below, the first output is a plain text file, and the second output is a json file. The text file is converted into a string, and the json file into a dictionary. 

In [ ]:
resp = wps.execute("multiple_outputs", inputs=[("count", "1")])
output, ref = resp.processOutputs
print(output.reference)
print(ref.reference)
output.writeToDisk("/tmp/output.txt")

In [ ]:
output = cli.multiple_outputs(1).get()[0]
print(output)
# as reference
output = cli.multiple_outputs(1).get(asobj=True)[0]
print(output)